In [1]:
%pip install "gurobipy>=10.0"

# Install other dependencies
%pip install numpy
%pip install scipy
%pip install pandas


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ['GRB_LICENSE_FILE'] = r'C:\Users\AKSHAY\gurobi.lic'


In [3]:
import gurobipy as gp 
from gurobipy import GRB

In [4]:
n=int(input())

crystals=[]
for _ in range(n):
    a,b,c=map(int,input().split())
    crystals.append((a,b,c))

for row in crystals:
    print(row)

 4
 10 20 100
 40 50 150
 25 35 200
 50 50 300


(10, 20, 100)
(40, 50, 150)
(25, 35, 200)
(50, 50, 300)


In [6]:
m=int(input())

mines=[]
for _ in range(m):
    a,b,c=map(int,input().split())
    mines.append((a,b,c))

for row in mines:
    print(row)

 3
 15 25 50
 70 60 75
 30 200 60


(15, 25, 50)
(70, 60, 75)
(30, 200, 60)


In [7]:
model=gp.Model("model1")

Set parameter Username
Set parameter LicenseID to value 2616846
Academic license - for non-commercial use only - expires 2026-01-31


In [8]:
w_model=gp.Model("newmodel")


In [9]:
x_values = w_model.addVars(1000, vtype=GRB.CONTINUOUS, name="x_coordinates",lb=0.0,ub=10000.00)
y_values = w_model.addVars(1000, vtype=GRB.CONTINUOUS, name="y_coordinates",lb=0.0,ub=10000.00)
w_model.update()


In [ ]:
n=w_model.addVar(vtype=GRB.BINARY,name="n")
m=w_model.addVar(vtype=GRB.BINARY,name="m")
w_model.update()

In [ ]:
w_model.remove(n)
w_model.remove(m)
w_model.update()

In [10]:

# Define binary helper variables
b_vars = w_model.addVars(1000, vtype=GRB.BINARY, name="b")

# Big-M constant (choose carefully based on the variable bounds)
M = 10000.0  

# Add constraints to enforce at least one equality
for i in range(1000):
    w_model.addConstr(x_values[i] - x_values[(i-1) % 1000] <= M * b_vars[i], name=f"x_constraint_upper_{i}")
    w_model.addConstr(x_values[(i-1) % 1000] - x_values[i] <= M * b_vars[i], name=f"x_constraint_lower_{i}")

    w_model.addConstr(y_values[i] - y_values[(i-1) % 1000] <= M * (1 - b_vars[i]), name=f"y_constraint_upper_{i}")
    w_model.addConstr(y_values[(i-1) % 1000] - y_values[i] <= M * (1 - b_vars[i]), name=f"y_constraint_lower_{i}")

# Update model
w_model.update()


In [11]:
def sfunc(i,j):
  
    ximi=x_values[(i-1)%1000]
    xjmi=x_values[(j-1)%1000]
    xi=x_values[i%1000]
    xj=x_values[j%1000]

    yimi=y_values[(i-1)%1000]
    yjmi=y_values[(j-1)%1000]
    yi=y_values[i%1000]
    yj=y_values[j%1000]

    #for s 
    
    s_num=((yjmi-yimi)*(xi-ximi))-((xjmi-ximi)*(yi-yimi))
    s_den=((xj-xjmi)*(yi-yimi))-((xi-ximi)*(yj-yjmi))

    s=s_num/s_den
    return s

def tfunc(i,j):

    ximi=x_values[(i-1)%1000]
    xjmi=x_values[(j-1)%1000]
    xi=x_values[i%1000]
    xj=x_values[j%1000]

    yimi=y_values[(i-1)%1000]
    yjmi=y_values[(j-1)%1000]
    yi=y_values[i%1000]
    yj=y_values[j%1000]

    #for s 
    
    t_num=((yjmi-yimi)*(xj-xjmi))-((xjmi-ximi)*(yj-yjmi))
    t_den=((xj-xjmi)*(yi-yimi))-((xi-ximi)*(yj-yjmi))

    t=t_num/t_den
    return t
    
    
        
        

In [12]:

# Define binary helper variables indexed by (i, j, type)
b_varss = w_model.addVars(998000 * 2, vtype=GRB.BINARY, name="b")

# Constants
MM = 10000  # Big-M constant
epsilon = 1e-6  

# Counter for binary variable index
cnt = 0  

# Add constraints
for i in range(1, 998):
    for j in range(i + 2, 1000):
        ximi = x_values[i - 1]
        xjmi = x_values[j - 1]
        xi = x_values[i]
        xj = x_values[j]

        yimi = y_values[i - 1]
        yjmi = y_values[j - 1]
        yi = y_values[i]
        yj = y_values[j]

        # Define auxiliary variable to avoid division
        num_x = (yjmi - yimi) * (xi - ximi) - (xjmi - ximi) * (yi - yimi)
        den_x = (xj - xjmi) * (yi - yimi) - (xi - ximi) * (yj - yjmi)

        num_y = (yjmi - yimi) * (xj - xjmi) - (xjmi - ximi) * (yj - yjmi)
        den_y = den_x  # Same denominator

        # Use Big-M to enforce strict inequalities
        w_model.addConstr(num_x <= -epsilon * den_x + MM * b_varss[cnt], name=f"x_lt_0_{i}_{j}")
        w_model.addConstr(num_x >= (1 + epsilon) * den_x - MM * (1 - b_varss[cnt]), name=f"x_gt_1_{i}_{j}")
        cnt += 1

        w_model.addConstr(num_y <= -epsilon * den_y + MM * b_varss[cnt], name=f"y_lt_0_{i}_{j}")
        w_model.addConstr(num_y >= (1 + epsilon) * den_y - MM * (1 - b_varss[cnt]), name=f"y_gt_1_{i}_{j}")
        cnt += 1

# Update model
w_model.update()


In [23]:
inside = model.addVars(n + m, vtype=GRB.BINARY, name="inside")

# Binary variables to count crossings for ray tracing
crossing_vars = model.addVars(n + m, 1000, vtype=GRB.BINARY, name="crossing")

M = 10000  # Large constant for big-M constraints

# Apply ray tracing for each crystal and mine
for idx, (a, b, c) in enumerate(crystals + mines):
    crossing_sum = gp.LinExpr(0)  # To count total crossings

    for i in range(1000):
        j = (i + 1) % 1000  # Next vertex in the polygon

        # Edge endpoints
        x1, y1 = x_values[i], y_values[i]
        x2, y2 = x_values[j], y_values[j]

        # Define binary variable crossing_vars[idx, i] = 1 if ray crosses edge (i, j)
        model.addConstr(crossing_vars[idx, i] <= (b - y1) * (b - y2) * (-1), name=f"crossing_cond1_{idx}_{i}")
        model.addConstr(crossing_vars[idx, i] <= (a - x1), name=f"crossing_cond2_{idx}_{i}")
        model.addConstr(crossing_vars[idx, i] >= (b - y1) * (b - y2) * (-1) + (a - x1) - 1, name=f"crossing_cond3_{idx}_{i}")

        # Sum up crossings
        crossing_sum += crossing_vars[idx, i]

    # Ensure `inside[idx]` captures the odd/even nature of crossings
    aux = model.addVar(vtype=GRB.INTEGER, name=f"aux_{idx}")
    model.addConstr(crossing_sum == 2 * aux + inside[idx])

# Compute total score
total_score = gp.LinExpr(0)

for i in range(n):
    total_score += inside[i] * crystals[i][2]

for i in range(m):
    total_score -= inside[n + i] * mines[i][2]


GurobiError: Variable not in model

In [ ]:
# Set objective
model.setParam(GRB.Param.IterationLimit, 1000)  # Stop after 10,000 iterations



model.setObjective(total_score, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Output the optimal score
if model.status == GRB.OPTIMAL:
    print(f"Optimal Score: {model.objVal}")
else:
    print("No optimal solution found.")